In [213]:
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import time

with open('jsonFiles/monsters.json') as f:
    data = json.load(f)

#pprint(data) # Prints out all of the Json File

numMonst = 325;
monstNames = np.chararray((numMonst,1))
monstNames = [None]*numMonst
monstCRs = np.zeros((numMonst,1))

# Monsters have up to 7 actions
# Monsters have up to 3 special abilities

for i in range(0,numMonst):
    monstNames[i] = data[i]["name"]
    monstCRs[i] = eval( data[i]["challenge_rating"] )
#     print monstNames[i]
#     print monstCRs[i]
    #print len(data[i]["special_abilities"])
    #print(data[i]["challenge_rating"])


def plot_hist_of_CRs():
    gaussian_numbers = np.random.randn(1000)
    plt.hist(monstCRs,10)
    plt.title("CR Histogram")
    plt.xlabel("CR")
    plt.ylabel("Frequency")
    plt.show()
    
def print_random_monst_data():
    randomMonstIndex = np.random.randint(numMonst);
    pprint(data[randomMonstIndex])
    i = randomMonstIndex
    
def print_specific_monst_data( index=0 ):
    i = index
    print ('******** '+data[i]['name']+' ******** ')
    print ('________________________________________')
    pprint(data[i])
    
def roll_dice( numDice, sides, modifier=0 ):
    total = modifier
    for i in range(numDice):
        total += numDice * (np.random.randint(sides) + 1)
    return total

def A_attack_B(attacker=None,defender=None ):
    self = attacker
    i = np.random.randint( self.num_actions );
    totalDamage = 0
    
    enemy_HP = defender.hit_points
    enemy_AC = defender.armor_class
    
    name = str( self.name )
    actionName = str( self.actions_name[i] )
    actionDesc = str( self.actions_desc[i] )
    attackBonus = int( self.actions_attack_bonus[i] )
    damageBonus = int( self.actions_damage_bonus[i] )
    
    attackRoll = int( self.actions_attack_bonus[i] )
    damage = int( self.actions_attack_bonus[i] )
    
    print('____________________________________________')
    print('________________A C T I O N_________________')
    print('____________________________________________')
    if actionName=='Multiattack':
        print( 'It Attacks Multiple Times!')
        print('The '+name+' uses its '+actionName)
        print('2x')
        print('Attack Bonus: '+str(attackBonus))
        print('Damage Bonus: '+str(damageBonus))
        for i in range (1,3):
            atackRoll = roll_dice( 1, 20, attackBonus )
            damage = roll_dice( 1, 6, damageBonus )
            print('Rolled a '+str(atackRoll))
            if attackRoll > enemy_AC:
                print('Attack landed!')
                print('Did '+str(damage)+' damage!')
                totalDamage += damage
            else:
                print('Attack missed!')
            
    elif attackBonus == 0 and damageBonus == 0:
        print('The '+name+' uses its '+actionName)
        print( actionDesc )
          
    else:
        print('The '+name+' uses its '+actionName)
        print( actionDesc )
        print('')
        print('Attack Bonus: '+str(attackBonus))
        print('Damage Bonus: '+str(damageBonus))
        attackRoll = roll_dice( 1, 20, attackBonus )
        damage = roll_dice( 1, 6, damageBonus )
        print('Rolled a '+str(attackRoll))
        if attackRoll > enemy_AC:
            print('Attack landed!')
            print('Did '+str(damage)+' damage!')
            totalDamage += damage
        else:
            print('Attack missed!')
        print('')
        
    defender.hit_points -= totalDamage
    #print('Has '+str(defender.hit_points)+' HP left!')
    print('____________________________________________')
    print('\n\n')
    

    
    
class Monster:
    kind = 'monster'         # class variable shared by all instances

    def __init__(self, is_random=False, index=None):
        
        if is_random:
            randomMonstIndex = np.random.randint(numMonst);
            i = randomMonstIndex
        elif index==None:
            i = 0
        elif index!=None:
            i = index
        else:
            i = 0
        
        self.canSwim = False
        self.canFly = False
        
        self.name = data[i]['name']    # instance variable unique to each instance
        self.typ = data[i]['type']
        self.subtype = data[i]['subtype']
        self.cr = data[i]['challenge_rating']
        self.alignment = data[i]['alignment']
        self.size = data[i]['size']
        self.index = monstNames.index(self.name)
        
        self.intelligence = data[i]['intelligence']
        self.wisdom = data[i]['wisdom']
        self.strength = data[i]['strength']
        self.dexterity = data[i]['dexterity']
        self.charisma = data[i]['charisma']
        self.constitution = data[i]['constitution']
        
        self.hit_dice = data[i]['hit_dice']
        self.hit_points = data[i]['hit_points']
        self.armor_class = data[i]['armor_class']

        self.speed = data[i]['speed']
        self.senses = data[i]['senses']
        self.languages = data[i]['languages']

        self.condition_immunities = data[i]['condition_immunities']
        self.damage_immunities = data[i]['damage_immunities']
        self.damage_resistances = data[i]['damage_resistances']
        self.damage_vulnerabilities = data[i]['damage_vulnerabilities']
        
        try: # Not all monsters have stealth
            self.stealth = data[i]['stealth']
        except Exception, e:
            #print repr(e)
            self.stealth = 0
            
        # ACTIONS
        self.num_actions = 0
        self.actions_attack_bonus = []
        self.actions_damage_bonus = []
        self.actions_desc = []
        self.actions_name = []
        self.actions_damage_dice = []
        try: # Not all monsters have actions
            self.num_actions = len(data[i]['actions'])
            self.actions = data[i]['actions']
            for j in range(0,self.num_actions):
                self.actions_attack_bonus.append(self.actions[j]['attack_bonus'])
                self.actions_desc.append(self.actions[j]['desc'])
                self.actions_name.append(self.actions[j]['name'])

                # Some special abilities do not have damage dice
                try:
                    self.actions_damage_dice.append(self.actions[j]['damage_dice'])
                except Exception, e:
                    self.actions_damage_dice.append('0d0')
                # Some special abilities do not have damage bonus
                try:
                    self.actions_damage_bonus.append(self.actions[j]['damage_bonus'])
                except Exception, e:
                    self.actions_damage_bonus.append(0)
                    
        except Exception, e:
            #print repr(e)
            #print self.name+' has NO ACTIONS'
            self.actions = None
        
        # SPECIAL ABILITIES
        self.num_special_abilities = 0
        self.special_abilities_name = []
        self.special_abilities_desc = []
        self.special_abilities_attack_bonus = []
        try: # Not all monsters have special abilities
            self.num_special_abilities = len(data[i]['special_abilities'])
            self.special_abilities = data[i]['special_abilities']
            for j in range(0,self.num_special_abilities):
                self.special_abilities_name.append(self.special_abilities[j]['name'])
                self.special_abilities_desc.append(self.special_abilities[j]['desc'])
                self.special_abilities_attack_bonus.append(self.special_abilities[j]['attack_bonus'])
        except Exception, e:
            #print repr(e)
            self.special_abilities = None
            
        if 'fly' in self.speed:
            self.canFly = True
        if 'swim' in self.speed:
            self.canSwim = True
            
    def perform_random_action(self, enemy_AC = 69):
        i = np.random.randint( self.num_actions );
        
        totalDamage = 0
        atackRoll = 0
        damage = 0
        
        name = str( self.name )
        actionName = str( self.actions_name[i] )
        actionDesc = str( self.actions_desc[i] )
        attackBonus = int( self.actions_attack_bonus[i] )
        damageBonus = int( self.actions_damage_bonus[i] )
        
        
    def perform_random_special_ability(self):
        
        
        if self.num_special_abilities == 0:
            print 'No special abilities!'
            print ''
        else:
            i = np.random.randint( self.num_special_abilities );

            name = str( self.name )
            specialAbilityName = str( self.special_abilities_name[i] )
            specialAbilityDesc = str( self.special_abilities_desc[i] )
            specialAbilityBonus = int( self.special_abilities_attack_bonus[i] )
            
            print('____________________________________________')
            print('_______________A B I L I T Y________________')
            print('____________________________________________')

            print('The '+name+' uses its '+specialAbilityName)
            print( specialAbilityDesc )
            print('')
            print('____________________________________________')

            

In [214]:
monst1 = Monster(is_random=True)
monst1_index = monst1.index

playerIndex = monstNames.index('Bandit Captain')
player = Monster( index=playerIndex )

turn = 1

print(player.name+' VS '+monst1.name)
print(str(player.armor_class)+' VS '+str(monst1.armor_class))
time.sleep(1.6)

while True:
    if player.hit_points < 1:
        print(monst1.name+' IS WINNER')
        break
    elif monst1.hit_points < 1:
        print(player.name+' IS WINNER')
        break
        
    
    A_attack_B( attacker=monst1, defender=player)
    
    #print('To attack push 0')
    #print('To deal push 0')
    #option = 0#input("What do you do? ")
    #if option == 0:
    A_attack_B( attacker=player, defender=monst1)
    time.sleep(1.6)
    #else:
    #    print('But Wai')
    
    turn += 1
    print('\n TURN '+str(turn))
    print('____________________________________________')
    print('Player has '+str(player.hit_points)+' HP')
    print(monst1.name+' has '+str(monst1.hit_points)+' HP')
    print('\n')
    

Bandit Captain VS Balor
15 VS 19
____________________________________________
________________A C T I O N_________________
____________________________________________
The Balor uses its Variant: Summon Demon (1/Day)
The demon chooses what to summon and attempts a magical summoning.
A balor has a 50 percent chance of summoning 1d8 vrocks, 1d6 hezrous, 1d4 glabrezus, 1d3 nalfeshnees, 1d2 mariliths, or one goristro.
A summoned demon appears in an unoccupied space within 60 feet of its summoner, acts as an ally of its summoner, and can't summon other demons. It remains for 1 minute, until it or its summoner dies, or until its summoner dismisses it as an action.
____________________________________________



____________________________________________
________________A C T I O N_________________
____________________________________________
The Bandit Captain uses its Scimitar
Melee Weapon Attack: +5 to hit, reach 5 ft., one target. Hit: 6 (1d6 + 3) slashing damage.

Attack Bonus: 5
Damag


 TURN 10
____________________________________________
Player has 24 HP
Balor has 262 HP


____________________________________________
________________A C T I O N_________________
____________________________________________
The Balor uses its Longsword
Melee Weapon Attack: +14 to hit, reach 10 ft., one target. Hit: 21 (3d8 + 8) slashing damage plus 13 (3d8) lightning damage. If the balor scores a critical hit, it rolls damage dice three times, instead of twice.

Attack Bonus: 14
Damage Bonus: 8
Rolled a 24
Attack landed!
Did 10 damage!

____________________________________________



____________________________________________
________________A C T I O N_________________
____________________________________________
It Attacks Multiple Times!
The Bandit Captain uses its Multiattack
2x
Attack Bonus: 0
Damage Bonus: 0
Rolled a 3
Attack missed!
Rolled a 15
Attack missed!
____________________________________________




 TURN 11
____________________________________________
Player has 14 

In [203]:
player.armor_class

15

In [ ]:
monst1 = Monster(is_random=True)
monst1_index = monst1.index
monst1.perform_random_action()
monst1.perform_random_special_ability()
